In [1]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.model_transformer import ModelTransformer
from transformer.date_transformer import DateTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [2]:
## Loading Constants
start = "2014-08-01"
end = datetime(2020,10,1).strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500")
market.close()
reload = False
ticker_range = range(len(tickers["Symbol"].unique()))
weekly_range = range(1,54)
weekly_range_final = range(1,54)
yearly_range = range(2015,2021)

In [4]:
## btc week conversions
btc = DateTransformer.convert_to_date(btc,"date")
btc["week"] = [row[1]["date"].week for row in btc.iterrows()]
btc["year"] = [row[1]["date"].year for row in btc.iterrows()]
btc.dropna()
btc_weekly_regression = btc.groupby(["year","week"]).mean().reset_index()
btc_weekly_classification = btc_weekly_regression.copy()
btc_weekly_classification["btc"] = btc_weekly_classification["btc"].pct_change()
btc_weekly_classification["btc"] = [1 if x > 0 else 0 for x in btc_weekly_classification["btc"]]
regression_data = btc_weekly_regression[["year","week","btc"]].merge(regression,on=["year","week"],how="left").dropna()[1:]
classification_data = btc_weekly_classification[["year","week","btc"]].merge(classification,on=["year","week"],how="left").dropna()[1:]

In [5]:
regression_data

,year,week,btc,A,AAL,AAP,AAPL,ABC,ABMD,ABT,...,XEL,XLNX,XOM,XRAY,XRX,YUM,ZBH,ZBRA,ZION,quarter
1,2014,38,416.859003,39.640679,35.379577,131.791716,22.983103,69.751999,25.096,38.209442,...,25.778725,38.162270,75.129002,44.730175,31.059266,45.986063,99.142167,74.461,26.425127,3.0
2,2014,39,407.692566,38.911727,33.674764,127.832338,22.858397,70.110780,24.970,37.548709,...,25.371179,37.971174,74.127344,44.010817,30.374798,45.813188,97.559123,73.470,26.126150,3.0
3,2014,40,361.426570,38.601443,33.086963,129.924333,22.653879,70.012930,24.638,36.688343,...,25.174298,36.764112,72.652767,43.904420,29.719774,45.672092,96.670933,69.700,25.731678,3.6
4,2014,41,355.234580,37.732785,31.144007,132.349236,22.715552,69.945886,23.392,36.985143,...,25.798030,35.103854,72.243421,43.630540,29.067189,44.510269,97.276604,67.374,24.971294,4.0
5,2014,42,390.479854,35.503182,29.887133,131.309148,22.221716,68.739078,22.822,36.024830,...,26.547501,33.361196,70.305228,43.286508,28.546028,43.205993,92.842645,63.149,23.586182,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,2020,42,11438.529018,105.586000,12.438000,156.544000,121.074881,98.288000,281.138,108.685869,...,72.860000,117.820526,34.310000,45.532000,20.058000,97.306000,146.830000,288.012,30.980000,4.0
320,2020,43,12645.532645,105.774000,12.771000,153.660000,116.029595,98.324000,278.304,107.920000,...,71.526000,114.121310,33.860000,46.493000,19.698000,98.946000,142.296000,296.578,31.250000,4.0
321,2020,44,13500.465960,103.288000,11.284000,148.256000,113.210464,96.476000,273.664,106.822000,...,71.010000,119.701044,32.666000,46.396000,17.940000,96.104000,135.484000,289.038,31.594000,4.0
322,2020,45,14727.649693,108.728000,11.301000,155.430000,114.219721,103.090000,265.340,111.304000,...,72.904000,125.954813,33.316000,47.185000,18.654000,98.136000,136.720000,325.604,32.629000,4.0


In [6]:
sims = []
test = True
training_days = 100
gap = 5
week_gap = int(gap/5)
training_years = 4
training_days = 100
daily_gap = 2
timeline = DateUtils.create_timeline(start,end)
sims = []
sec.connect()
strat_db.connect()
market.connect()
classification = classification_data
regression = regression_data
strat_db.drop_table("btc_weekly_sim")
for year in tqdm(yearly_range):
    for week in tqdm(weekly_range):
        try:
            ## Setting Up
            for i in ticker_range:
                try:
                    ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                    if ticker in regression.columns:
                        price = market.retrieve_price_data("prices",ticker)
                        mt = ModelTransformer(ticker,"","",week_gap)
                        mr = ModelPreprocessor(ticker)
                        sp = Modeler(ticker)
                        ## regression_model
                        week_index = regression[(regression["year"] == year) & (regression["week"] == week)].index.values.tolist()[0]
                        training_data = regression[:week_index].copy()[["btc",ticker]]
                        regression_prediction_data = regression.iloc[week_index-1].copy()
                        training_data["y"] = training_data[ticker].shift(-week_gap)
                        training_data = training_data[:-gap]
                        qpd = mr.day_trade_preprocess_regression(training_data.copy(),ticker)
                        q1c = qpd["X"].columns
                        rpr = sp.quarterly_model("prices",qpd,str(training_years),str(week_gap))
#                         ## classification_model
                        week_index = classification[(classification["year"] == year) & (classification["week"] == week)].index.values.tolist()[0]
                        training_data = classification[:week_index].copy()[["btc",ticker]]
                        classification_prediction_data = classification.iloc[week_index-1].copy()
                        training_data["y"] = training_data[ticker].shift(-week_gap)
                        training_data = training_data[:-gap]
                        qpd = mr.day_trade_preprocess_classify(training_data.copy(),ticker)
                        rpr = sp.quarterly_model("prices",qpd,str(training_years),str(week_gap))
                        cpr = sp.classify_wta("prices",qpd,str(training_years),str(week_gap))
                        price_results = pd.DataFrame([cpr,rpr])
                        product_qpds = []
                        for i in range(len(price_results)):
                            price_result = price_results.iloc[i]
                            if price_result["classification"] == False:
                                weekly_price_data = regression_prediction_data
                                weekly_price_data["y"] = weekly_price_data[ticker]
                            else:
                                weekly_price_data = classification_prediction_data
                                weekly_price_data["y"] = weekly_price_data[ticker]
                            pqpd = {"X":[weekly_price_data[["btc",ticker]]],"Y":weekly_price_data["y"]}
                            product_qpds.append(pqpd)
                        else:
                            product_dates = timeline[(timeline["year"] == year) & (timeline["week"] == week)]
                            start_prod = str(product_dates.head(1)["date"].item()).split(" ")[0]
                            end_prod = str(product_dates.tail(1)["date"].item()).split(" ")[0]
                            prot = ProductTransformer(ticker,start_prod,end_prod)
                            sim = prot.merge_btc_weeklies(price.copy(),product_qpds,price_results)
                            if len(sim) > 0:
                                strat_db.store_data("btc_weekly_sim",sim)
                except Exception as e:
                    message = {"status":"weekly modeling","ticker":ticker,"year":str(year),"week":str(week),"message":str(e)}
                    print(message)
        except Exception as e:
            print(year,week,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                   | 0/53 [00:00<?, ?it/s]

{'status': 'weekly modeling', 'ticker': 'YUM', 'year': '2015', 'week': '1', 'message': "'date'"}
{'status': 'weekly modeling', 'ticker': 'ZBRA', 'year': '2015', 'week': '1', 'message': "'date'"}



  2%|██▎                                                                                                                      | 1/53 [01:36<1:23:13, 96.04s/it]

{'status': 'weekly modeling', 'ticker': 'ZBH', 'year': '2015', 'week': '1', 'message': "'date'"}
{'status': 'weekly modeling', 'ticker': 'ZION', 'year': '2015', 'week': '1', 'message': "'date'"}
{'status': 'weekly modeling', 'ticker': 'YUM', 'year': '2015', 'week': '2', 'message': "'date'"}
{'status': 'weekly modeling', 'ticker': 'ZBRA', 'year': '2015', 'week': '2', 'message': "'date'"}
{'status': 'weekly modeling', 'ticker': 'ZBH', 'year': '2015', 'week': '2', 'message': "'date'"}



  4%|████▌                                                                                                                    | 2/53 [03:12<1:21:51, 96.30s/it]

...

{'status': 'weekly modeling', 'ticker': 'SJM', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'JNJ', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'JCI', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'JPM', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'JNPR', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'KSU', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'K', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'KEY', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'KMB', 'year': '2020', 'week': '53', 'mes

{'status': 'weekly modeling', 'ticker': 'ORLY', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'OXY', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'ODFL', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'OMC', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'OKE', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'ORCL', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'PCAR', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'PKG', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'PAYX', 'year': '2020', 'week': '53'

{'status': 'weekly modeling', 'ticker': 'TEL', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'FTI', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TDY', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TFX', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TER', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TXN', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TXT', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TMO', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'TIF', 'year': '2020', 'week': '53', 'me


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [14:18:24<00:00, 8584.13s/it]

{'status': 'weekly modeling', 'ticker': 'XRX', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'XLNX', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'YUM', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'ZBRA', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'ZBH', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
{'status': 'weekly modeling', 'ticker': 'ZION', 'year': '2020', 'week': '53', 'message': 'list index out of range'}
